<h1>Pacakages</h1>

In [1]:
import pandas as pd


from tqdm.notebook import tqdm
tqdm.pandas()

from utils import full_clean, create_t3_df, create_t1_df, create_complete_df, confusion_matrix, get_type_table, get_type_df

<h1>Clean the bot</h1>

In [2]:
bot_list = pd.read_csv('/Users/victor/Desktop/CS/UST_summer/bot_list.csv')
bot_list['Username'] = bot_list.Username.apply(lambda x: x[3:])

In [3]:
popular_author = pd.read_csv('/Users/victor/Desktop/CS/UST_summer/10k_sub_final.csv')
popular_bot = popular_author[popular_author.is_bot == 1]

In [4]:
fp = '/Users/victor/Desktop/CS/UST_summer/extracted_cols.csv'
selected_cols = ['author', 'id', 'link_id', 'parent_id']
extracted_cols = pd.read_csv(fp, low_memory = False)[selected_cols]

bot_extracted_cols, extracted_cols = full_clean(extracted_cols, bot_list, popular_bot, by_parent=True)

  0%|          | 0/32874876 [00:00<?, ?it/s]

In [5]:
# save extracted_cols bot
bot_extracted_cols.to_csv('bot_extracted_cols_final.csv')

In [6]:
fp = '/Users/victor/Desktop/CS/UST_summer/all_posts_clean_1.csv' # the directory of the raw data file, pipeliend from reddits
sc = ['id', 'author','title','created_utc']
all_posts = pd.read_csv(fp, low_memory=False, encoding='utf-8')[sc]

all_posts_bot, all_posts_df = full_clean(all_posts, bot_list, popular_bot)

  0%|          | 0/1087302 [00:00<?, ?it/s]

In [7]:
# save bot df
all_posts_bot.to_csv('all_posts_bot_final.csv')

In [8]:
all_posts_df.to_csv('all_posts_df_final.csv')

<h1>For t3-df</h1>

In [9]:
t3_name_id, t3_df = create_t3_df(extracted_cols)

  0%|          | 0/356800 [00:00<?, ?it/s]

In [10]:
# save the t3_df
t3_df.to_csv('t3_df_final.csv')

<h1>For t1-df</h1>

In [11]:
t1_name_id, t1_df = create_t1_df(extracted_cols)

  0%|          | 0/12112219 [00:00<?, ?it/s]

In [12]:
# save the t1_df
t1_df.to_csv('t1_df_final.csv')

<h1>Form the complete dataframe</h1>

In [13]:
complete_df = create_complete_df(t1_name_id, t1_df, t3_name_id, t3_df)

In [25]:
complete_df.head()

,parent_id,author,sub_comments,main_post,parent_author
0,t1_h0csj8y,"[Optionsfuckup, jackthemackattack, WeHaveBeenF...",2289,0,NaN
1,t1_gl35xpy,"[chayochan, thats_not_funny_guys, Appearance-R...",2210,0,NaN
2,t1_golywrq,"[Obliterateee, Grizzlytash82, MrProMuch, Squea...",1303,0,NaN
3,t1_fkifvew,"[noemojica88, emenpy, mycochang, datastrm, gar...",1204,0,NaN
4,t1_gkqb2tu,"[SilentSakura, onlybreakingbad, Content-Necess...",1062,0,SoDakZak


In [14]:
complete_df.to_csv('complete_df_final.csv')

<h1>For Confusion matrix</h1>

In [15]:
# confusion matrix
no_idea_comment, main_post_comment, comment_comment = confusion_matrix(complete_df)

<h1> For type table</h1>

In [16]:
# get the type table
type_table, post_only, post_comment, main_post_comment_only, comment_comment_only, no_idea_comment_only = get_type_table(complete_df, no_idea_comment, main_post_comment, comment_comment)

In [17]:
# save the type table
type_table.to_csv('type_table_final.csv')

<h1>For type dataframe</h1>

In [18]:
type_df = get_type_df(post_only, post_comment, main_post_comment_only, comment_comment_only, no_idea_comment_only)

In [19]:
# save the type_dataframe
type_df.to_csv('type_df_final.csv')

<h1>Concat the bot</h1>

In [20]:
t3_bot = pd.read_csv('/Users/victor/Desktop/CS/UST_summer/all_posts_bot_final.csv')
extracted_cols_bot = pd.read_csv('/Users/victor/Desktop/CS/UST_summer/bot_extracted_cols_final.csv')

all_bot = pd.concat([t3_bot, extracted_cols_bot])
all_bot = all_bot.drop_duplicates()
all_bot.head()

,Unnamed: 0,id,author,title,created_utc,is_bot,link_id,parent_id
0,854,mhv3i6,Tea_Bot278,c'mon fortnite,1.617283e+09,True,NaN,NaN
1,3546,mioer6,Your_wifes_bfNOTaBOT,I think my stonks are broke not moving up or d...,1.617383e+09,True,NaN,NaN
2,17089,mpeebo,Con_Bot_,GEO short,1.618236e+09,True,NaN,NaN
3,31579,muma0v,ShotBot,DONT FORGET CHAMATH EVIL SCAM GOBLIN,1.618909e+09,True,NaN,NaN
4,45598,n0mhc6,BotDamian,"How to start with trading? Books, Websites, Ex...",1.619638e+09,True,NaN,NaN


In [21]:
all_bot.to_csv('all_bot_final.csv')

In [ ]:
def count(dataframe):
    count_df = dataframe.groupby(dataframe.author.tolist()).size().reset_index().rename(columns={0:'count'})
    
    count_df = count_df.drop_duplicates()
    
    return count_df['author', 'count']